In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.2MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 1.2MB 17.0MB/s 
     |████████████████████████████████| 194kB 50.7MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
Mounted at /content/gdrive


In [2]:
from fastai.vision.widgets import *
from random import uniform
import numpy as np
from scipy.stats import norm,norminvgauss
import math  
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Programa
Este programa encuentra la probabilidad de incumplimiento basado en el modelo Vasicek
Los parametros requeridos son lgd (perdida dada por defecto) k (capital requerido o  solo capital simple)
% hallazgo implica probabilidad predeterminada (iDP)
resolución de ecuaciones no lineales de base de probabilidad implícita en el modelo de Vasicek y capital requerido

# Dataset
Subimos el dataset, estableciendo el path y la ruta

In [3]:
path="/content/gdrive/My Drive/Kaggle"


In [4]:
df = pd.read_csv(path+"/Matlab/CSVTEST.csv", low_memory=False)


Tomamos el numero de filas del dataset

In [9]:
t=df.shape[0]

Extraemos el dataset como arrays

In [10]:
lgd = np.array(df["lgd"]).reshape((t,1))
loan = np.array(df["loan"]).reshape((t,1))
cntgn = np.array(df["cntgn"]).reshape((t,1))
iPD = np.array(df["iPD"]).reshape((t,1))
NoS = 1000000

Corremos la función

In [11]:
fLossDistSim(iPD, lgd, loan, cntgn,NoS)

array([[3.03071303e+05],
       [2.03584876e+02]])

# Funcion Rho Ecuacion 10
Aplica para cierto tipo de exposiciones crediticias, pero suponemos que toda la cartera crediticia
es de este tipo de créditos.

In [6]:
def frho(iPD):
    rho = .12 * (1 - np.exp(-50 * iPD) / (1 - np.exp(-50))) + .24 * (1 - (1 - np.exp(-50 * iPD) / (1 - np.exp(-50))))
    return rho

# Funcion fLossDistSim


In [34]:
def fLossDistSim(iPD, lgd, loan, cntgn,NoS):

    global df
    x = np.random.rand(1, NoS)
    rho = frho(iPD)
# VaR999 - value at risk 99.9% valor esperado al 99.9
    var999 = lgd * norm.cdf((np.sqrt(1 / (1 - rho))) * norm.ppf(iPD, 0, 1) + (np.sqrt(rho / (1 - rho))) *norm.ppf(0.999, 0, 1), 0, 1)
# simulated loans LGD * 
    L = lgd * norm.cdf((np.sqrt(1 / (1 - rho))) * norm.ppf(iPD, 0, 1) + (np.sqrt(rho / (1 - rho))) * norm.ppf(x, 0, 1), 0, 1)
  
    
    L1 = L * loan
    L0 = L1 - var999 * loan
    L3 = var999 * loan *lgd
    L2 = iPD * lgd * loan
    formula7=L3-L2
    ##total loan pd Avg LGD
    ##Formula 7 Capital
#     L0 = L1 - var999 * loan 
    
    #Expected Loss
# #     plt.hist(L.flatten(), bins=2000)
# #     plt.show()
#     #Cuanto puedes perder fuera del tail Expected Value
#     ind=np.transpose(np.where(L>var999,L,0))
#     ind=ind[(ind>0)]
# #     plt.hist(ind.flatten(), bins=2000)
# #     plt.show()
    F=formula7
    df=df.assign(F=F)
    df.to_csv ("t.csv", index = False, header=True)

# Produccion
Permitimos subir el dataframe

In [37]:
btn_upload = widgets.FileUpload()
btn_upload

FileUpload(value={}, description='Upload')

In [38]:
with open("CSVTEST.csv", "w+b") as i:
    i.write(btn_upload.data[0])

In [39]:
df = pd.read_csv("CSVTEST.csv", low_memory=False)

In [40]:
btn_run = widgets.Button(description='Clasificar')
btn_run

Button(description='Clasificar', style=ButtonStyle())

También necesitaremos un * controlador de eventos de clic *; es decir, una función que se llamará cuando se presione. Podemos simplemente copiar las líneas de código de arriba:

In [41]:
def on_click_classify(change):
    with open("CSVTEST.csv", "w+b") as i:
      i.write(btn_upload.data[0])
    df = pd.read_csv("CSVTEST.csv", low_memory=False)
    t=df.shape[0]
    lgd = np.array(df["lgd"]).reshape((t,1))
    loan = np.array(df["loan"]).reshape((t,1))
    cntgn = np.array(df["cntgn"]).reshape((t,1))
    iPD = np.array(df["iPD"]).reshape((t,1))
    NoS = 1000000
    fLossDistSim(iPD, lgd, loan, cntgn,NoS)

btn_run.on_click(on_click_classify)

In [36]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila

     |████████████████████████████████| 2.5MB 7.9MB/s 
     |████████████████████████████████| 552kB 45.8MB/s 
     |████████████████████████████████| 245kB 50.4MB/s 
     |████████████████████████████████| 112kB 26.8MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 430kB 47.3MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7667 sha256=84592a665f1060ed3bf46c29934392e55d46a2d6c4a641142feca26b9b867e4f
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built contextvars
ERROR: nbdev 1.1.12 has requirement nbconvert<6, but you'll have nbconvert 6.0.7 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 6.1 which is incompatible.
  Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.

Enabling: voila
- Writing config: /usr/etc/jupyter
    - Validating...
Error loading server extension voila
      X is voila importable?
